In [1]:
%matplotlib inline
import plotly
import pandas as pd
import seaborn as sns
import geopandas as gpd
import plotly.express as px
import ipywidgets as widgets

from shapely import wkt
from tabulate import tabulate
from sklearn import datasets
from dask import dataframe as dd
from shapely.geometry import Point
from matplotlib import pyplot as plt
from ipywidgets import interact, widgets
from matplotlib.widgets import Slider
from IPython.display import display
import numpy as np

# Read the CSV file into a DataFrame
path_data = '../data/'
path_img = '../images/'
crashes = pd.read_csv(path_data + 'crashes.csv')

# Display the DataFrame
crashes.head()  # This will display the first few rows of the DataFrame






,crash_date,crash_time,latitude,longitude,borough,region
0,2021-12-14,12:54,40.687534,-73.977500,Brooklyn,Fort Greene
1,2021-12-14,16:25,40.784615,-73.953964,Manhattan,Upper East Side North
2,2022-04-24,15:35,40.767242,-73.986206,Manhattan,Clinton East
3,2021-12-09,20:20,40.592070,-73.962990,Brooklyn,Homecrest
4,2021-12-09,23:15,40.640835,-73.989670,Brooklyn,Borough Park


In [2]:
regression_dataset = crashes[['crash_time', 'latitude', 'longitude']]
regression_dataset.head()
# regression_dataset['crash_time'].dtype

,crash_time,latitude,longitude
0,12:54,40.687534,-73.977500
1,16:25,40.784615,-73.953964
2,15:35,40.767242,-73.986206
3,20:20,40.592070,-73.962990
4,23:15,40.640835,-73.989670


In [3]:
regression_dataset['accident'] = 1
len(regression_dataset)
regression_dataset.head()

/tmp/ipykernel_19099/1429853167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_dataset['accident'] = 1


,crash_time,latitude,longitude,accident
0,12:54,40.687534,-73.977500,1
1,16:25,40.784615,-73.953964,1
2,15:35,40.767242,-73.986206,1
3,20:20,40.592070,-73.962990,1
4,23:15,40.640835,-73.989670,1


In [4]:
import geopandas as gpd
from shapely.geometry import Point, MultiPolygon
import random
import pandas as pd
from shapely.strtree import STRtree

# Load GeoJSON multipolygon
geojson_path = 'NYC_zone.geojson'
gdf = gpd.read_file(geojson_path)
multipolygon = gdf.geometry.unary_union  # Combine all polygons into one multipolygon if necessary

# Load positive samples
# positive_samples_path = 'path_to_your_positive_samples.csv'
positive_samples = regression_dataset

# Extract crash points and times
positive_points = [Point(xy) for xy in zip(positive_samples.longitude, positive_samples.latitude)]
positive_times = set(positive_samples.crash_time)  # Assuming crash_time is in a comparable format



In [5]:
# Create a spatial index for the multipolygon
tree = STRtree([multipolygon])

# Generate random points within the multipolygon
def generate_random_points_within_multipolygon(multipolygon, num_points):
    minx, miny, maxx, maxy = multipolygon.bounds
    points = []
    while len(points) < num_points:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if tree.query(pnt):  # Check if the point is within the multipolygon using the spatial index
            points.append(pnt)
    return points

# Check if a point is a positive sample
def is_positive_sample(point, time, positive_points, positive_times):
    return point in positive_points and time in positive_times

# Generate negative samples
num_negative_samples = len(positive_samples)  # Generating as many negative samples as positive ones
negative_samples = []

batch_size = 1000  # Number of points to generate in each batch
while len(negative_samples) < num_negative_samples:
    remaining_samples = num_negative_samples - len(negative_samples)
    print(remaining_samples)
    points = generate_random_points_within_multipolygon(multipolygon, min(batch_size, remaining_samples))
    
    for point in points:
        time = random.choice(positive_times)  # Randomly choose a time from positive samples
        if not is_positive_sample(point, time, positive_points, positive_times):
            negative_samples.append((point.y, point.x, time, 0))
            if len(negative_samples) >= num_negative_samples:
                break



26408


KeyboardInterrupt: 

In [6]:
# Create a DataFrame for negative samples
negative_samples_df = pd.DataFrame(negative_samples, columns=['latitude', 'longitude', 'crash_time', 'accident'])

# Save negative samples to CSV
negative_samples_df.to_csv('negative_samples.csv', index=False)

print("Negative samples generated and saved to 'negative_samples.csv'")

Negative samples generated and saved to 'negative_samples.csv'


In [18]:
# # Generate random points within the multipolygon
# def generate_random_point_within_multipolygon(multipolygon):
#     minx, miny, maxx, maxy = multipolygon.bounds
#     while True:
#         pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
#         if multipolygon.contains(pnt):
#             return pnt

# # Check if a point is a positive sample
# def is_positive_sample(point, time, positive_points, positive_times):
#     return point in positive_points and time in positive_times

# # Generate negative samples
# num_negative_samples = len(positive_samples)  # Generating as many negative samples as positive ones
# negative_samples = []

# cnt = 0
# while len(negative_samples) < num_negative_samples:
#     print(len(negative_samples), num_negative_samples)
    
#     point = generate_random_point_within_multipolygon(multipolygon)
#     time = random.choice(positive_samples.crash_time.unique())  # Randomly choose a time from positive samples
    
#     if not is_positive_sample(point, time, positive_points, positive_times):
#         negative_samples.append((point.y, point.x, time))

# # Create a DataFrame for negative samples
# negative_samples_df = pd.DataFrame(negative_samples, columns=['latitude', 'longitude', 'time'])

# # Save negative samples to CSV
# negative_samples_df.to_csv('negative_samples.csv', index=False)

# print("Negative samples generated and saved to 'negative_samples.csv'")


0 26408
1 26408
2 26408
3 26408
4 26408
5 26408
6 26408
7 26408
8 26408
9 26408
10 26408
11 26408
12 26408
13 26408
14 26408
15 26408
16 26408
17 26408
18 26408
19 26408
20 26408
21 26408
22 26408
23 26408
24 26408
25 26408
26 26408
27 26408
28 26408
29 26408
30 26408
31 26408
32 26408
33 26408
34 26408
35 26408
36 26408
37 26408
38 26408
39 26408
40 26408
41 26408
42 26408
43 26408
44 26408
45 26408
46 26408
47 26408
48 26408
49 26408
50 26408
51 26408
52 26408
53 26408
54 26408
55 26408
56 26408
57 26408
58 26408
59 26408
60 26408
61 26408
62 26408
63 26408
64 26408
65 26408
66 26408
67 26408
68 26408
69 26408
70 26408
71 26408
72 26408
73 26408
74 26408
75 26408
76 26408
77 26408
78 26408
79 26408
80 26408
81 26408
82 26408
83 26408
84 26408
85 26408
86 26408
87 26408
88 26408
89 26408
90 26408
91 26408
92 26408
93 26408
94 26408
95 26408
96 26408
97 26408
98 26408
99 26408
100 26408
101 26408
102 26408
103 26408
104 26408
105 26408
106 26408
107 26408
108 26408
109 26408
110 26408


KeyboardInterrupt: 

In [6]:
min_longitude, max_longitude = -74.3, -73.6
min_latitude, max_latitude = 40.4, 40.9

# Number of negative examples to generate
num_samples = 26408

# Generate random longitude and latitude coordinates within the bounding box
random_longitudes = np.random.uniform(min_longitude, max_longitude, num_samples)
random_latitudes = np.random.uniform(min_latitude, max_latitude, num_samples)

# Generate a random not-crash time (hours:minutes)
def generate_random_time():
    hour = np.random.randint(0, 24)
    minute = np.random.randint(0, 60)
    return f"{hour:02d}:{minute:02d}"

random_non_crash_time = [generate_random_time() for _ in range(num_samples)]

# Create DataFrame for negative examples
negative_examples = pd.DataFrame({
    'longitude': random_longitudes,
    'latitude': random_latitudes,
    'crash_time': random_non_crash_time,  # Assuming hour_of_day is a feature
    'accident': 0  # Label indicating no accident occurred
})

# Display the negative examples DataFrame
print(negative_examples.head())

   longitude   latitude crash_time  accident
0 -74.220413  40.653761      06:33         0
1 -73.608479  40.676047      23:00         0
2 -73.748660  40.835920      16:24         0
3 -74.069733  40.796839      13:50         0
4 -74.201095  40.853782      16:57         0


In [7]:
from sklearn.utils import shuffle
regression_dataset =  pd.concat([regression_dataset, negative_examples])

# Shuffle the combined DataFrame
regression_dataset = shuffle(regression_dataset).reset_index(drop=True)

# Display the shuffled DataFrame
print(regression_dataset)

      crash_time   latitude  longitude  accident
0           7:49  40.816746 -73.929040         1
1          20:10  40.497397 -73.768028         0
2          16:00  40.760723 -73.872086         1
3          14:45  40.499365 -73.939240         0
4          22:50  40.618797 -73.983500         1
...          ...        ...        ...       ...
52811      15:29  40.527821 -73.747701         0
52812      00:03  40.854149 -73.708024         0
52813       2:06  40.680756 -73.813446         1
52814      14:40  40.825745 -73.858430         1
52815      14:19  40.777412 -73.978806         1

[52816 rows x 4 columns]


In [11]:
def time_to_float(time_str):
    # Split the time string into hours and minutes
    hours, minutes = map(int, time_str.split(':'))
    # Convert hours and minutes to a single float value representing the time in hours
    return hours + minutes / 60


regression_dataset['crash_time'] = regression_dataset['crash_time'].apply(time_to_float)


AttributeError: 'float' object has no attribute 'split'

In [9]:
regression_dataset.to_csv('regression_dataset.csv', index=False)

print("Regresion dataset generated and saved to 'regression_dataset.csv'")

Negative samples generated and saved to 'negative_samples.csv'


In [41]:
# Step 1: Data Preparation
# Example of creating some sample data


# Step 2: Feature Engineering (if needed)
# Example: Extract hour of the day from the time column

# Step 3: Split data into train and test sets
X = regression_dataset[['longitude', 'latitude', 'crash_time']]
y = regression_dataset['accident']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Step 4: Model Training
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 5: Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Step 6: Prediction for specific points and times
# Assuming you have new points and times stored in X_new
X_new = pd.DataFrame({'longitude': [-74.0060, -118.2437], 'latitude': [40.7128, 34.0522], 'crash_time': ['10:12', '15:14']})
X_new['crash_time'] = X_new['crash_time'].apply(time_to_float)
predicted_probabilities = model.predict_proba(X_new)[:, 1]  # Probability of accident
print("Predicted Probabilities:", predicted_probabilities)


predicted_probabilities_in_percent = [prob * 100 for prob in predicted_probabilities]
print("Predicted Probabilities Per:", predicted_probabilities_in_percent)

Accuracy: 0.6677394926164332
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.61      0.65      2644
           1       0.65      0.72      0.69      2638

    accuracy                           0.67      5282
   macro avg       0.67      0.67      0.67      5282
weighted avg       0.67      0.67      0.67      5282

Predicted Probabilities: [4.66947859e-01 3.67127713e-23]
Predicted Probabilities Per: [46.69478587568659, 3.6712771336850615e-21]
